In [1]:
from sklearn.utils import shuffle
import pandas as pd
import joblib
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split,GridSearchCV
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import warnings

In [2]:
def XGBoost(X_train, X_test, y_train, y_test, X_predict, y_predict, fetures, loop, testindex):
    Xgb = XGBRegressor()
    
    #best parameter finding using GridsearchCV
    parameters = [
        {'n_estimators': [20,50,70], 'max_depth':[3,4,5,7,10]}]
    clf1 = GridSearchCV(Xgb, parameters, scoring='neg_mean_squared_error', cv=5)
    clf1.fit(X_train, y_train)
    param = clf1.best_params_
    #print params
    svr1 = XGBRegressor(n_estimators=param.get("n_estimators"), max_depth=param.get("max_depth"))
    svr1.fit(X_train,y_train)
    
    y_pred=svr1.predict(X_test)
    y_train_pred=svr1.predict(X_train)


    dftrainname="%d_XGB_train.csv"%(loop)
    dftestname="%d_XGB_test.csv"%(loop)

    result_train_df=pd.DataFrame()
    result_train_df["y_train"]=y_train
    result_train_df["y_train_pred"]=list(y_train_pred.reshape(1,-1)[0])
    
    
    result_test_df=pd.DataFrame()
    result_test_df["y_test"]=y_test
    result_test_df["y_pred"]=list(y_pred.reshape(1,-1)[0])
    #result_test_df["Index"]=testindex
    result_train_df.to_csv("./%s"%(dftrainname),index=False)
    result_test_df.to_csv("./%s"%(dftestname),index=False)
 
    dfpredictname="%d_XGB_predict.csv"%(loop)
    y_predict_predict=svr1.predict(X_predict) 
    result_predict_df=pd.DataFrame()
    #result_test_df["y_predict"]=y_predict
    result_predict_df["Index"]=testindex
    #result_test_df["Index"]=testindex
    result_predict_df["y_predict_predict"]=list(y_predict_predict.reshape(1,-1)[0])
    result_predict_df.to_csv("./%s"%(dfpredictname),index=False)


    train_score = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_score = np.sqrt(mean_squared_error(y_test, y_pred))
    test_r2=r2_score(y_test, y_pred)
    train_r2=r2_score(y_train, y_train_pred)

    train_mae=mean_absolute_error(y_train, y_train_pred)
    test_mae=mean_absolute_error(y_test, y_pred)
    
    joblib.dump(svr1,'%s_XGB_all.model'%(loop))

    print("XGB",train_r2 ,test_r2, train_score, test_score, train_mae, test_mae)
    return train_r2, test_r2, train_score, test_score, train_mae, test_mae 


In [ ]:
def loops(num=10):    
    #warnings.simplefilter('ignore')
    rbf_array=[]
    linear_array=[]
    rf_array=[]
    XGB_array=[]
    loopindex=[]

    for loop in range(1,num+1):
        print("loop %d... " %(loop))
        data=pd.read_csv(r"./H111.csv")
        testdata=pd.read_csv(r"./H3876_Hypothetical.csv")
       
        data = shuffle(data)

        df_corr=data.corr()
        df_corr.to_csv(r"df_corr.csv")

        
        data.drop(["Index"],axis=1,inplace=True)
        
        #print("Testdata Index")
        testindex=testdata["Index"] ####################
        #print(testindex)
        testdata.drop(["Index"],axis=1,inplace=True)

        y=data["Hv30"]
        X=data.drop("Hv30",axis=1)
        #y_test=testdata["Hv30"]
        #X_test=testdata.drop("Hv30",axis=1)


        X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)
        #print(y_train)
        std=MinMaxScaler()
        X_train=std.fit_transform(X_train)
        
        X_test=std.transform(X_test) #STANDARDISING test inputs
        
        y_predict=testdata["Hv30"]
        X_predict=testdata.drop("Hv30",axis=1)
                                                #Standardize the values
        X_predict=std.transform(X_predict)

        features = list(X.columns)


        loopindex.append(int(loop))
        

        XGB_record=[]
        XGBflisttmp=list(XGBoost(X_train, X_test, y_train, y_test, X_predict, y_predict, features, loop, testindex))
        XGB_record.extend(rflisttmp)
        XGB_array.append(XGB_record)

    
    XGB_array_result=np.array(XGB_array)
    XGB_array_mean=["mean_XGB"]
    XGB_array_mean.extend(list(np.mean(XGB_array_result, axis=0)))
    XGB_array_mean.extend(["NAN", "NAN"])
    XGB_array_std=["std_rf"]
    XGB_array_std.extend(list(np.std(XGB_array_result, axis=0)))
    XGB_array_std.extend(["NAN", "NAN"])
    XGB_array_result_6=XGB_array_result[:,1]
    XGB_test_mae_mean=np.mean(XGB_array_result, axis=0)[1]
    loopindex_array=np.array(loopindex).reshape(-1,1)
    loopindex_array.dtype=int
    error_XGB=XGB_array_result_6.reshape(-1,1) - XGB_test_mae_mean.reshape(-1,1)
    error_XGB_absolute=np.absolute(error_XGB)
    XGB_data_all=np.concatenate((loopindex_array,XGB_array_result, error_XGB_absolute), axis=1)
    XGB_data_all_sort=np.concatenate((XGB_data_all[np.lexsort(XGB_data_all.T)],loopindex_array), axis=1)
    title_XGB=["XGB_loop" ,"XGB_train_r2", "XGB_test_r2", "XGB_train_rmse", "XGB_test_rmse", "XGB_train_mae", "XGB_test_mae", "XGB_mae_error", "XGB_mae_error_index"]
    XGB_data_mean_std=np.concatenate((np.array([title_XGB]) ,XGB_data_all_sort, np.array([XGB_array_mean]), np.array([XGB_array_std])), axis=0)

   
    XGB=np.concatenate((XGB_data_mean_std), axis=0)

    columns=["loop" ,"train_r2", "test_r2", "train_rmse", "test_rmse", "train_mae", "test_mae", "mae_error", "mae_error_index"]
    scoreing=pd.DataFrame(XGB, columns=columns)
    scoreing.to_csv(r"./scores.csv", index=False)

    np.set_printoptions(precision=5)


if __name__ == '__main__':
    loops(10)
    
